In [6]:
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
import json
import os
from typing import Annotated, Dict

import autogen
from autogen import AssistantAgent, UserProxyAgent, register_function
from autogen.cache import Cache
from typing import Annotated
from typing import Annotated
from openai import AzureOpenAI
from typing import Annotated
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.identity import AzureCliCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
    AzureOpenAIVectorizer,
    AzureOpenAIParameters
)

load_dotenv()
# Configure environment variables
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION="2024-02-01"
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AISTUDIO_OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("AISTUDIO_OPENAI_GPT4_DEPLOYMENT_NAME")
azure_openai_embedding_dimensions = 1536
index_name = "books"



In [7]:
# Configure OpenAI API
aoai_client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_API_KEY,  
  api_version=AZURE_OPENAI_API_VERSION
)


llm_config = {
    "cache_seed": 43,  # change the cache_seed for different trials
    "temperature": 0,
    "timeout": 120,  # in seconds
    "config_list": 
    [
        {
            "model": AZURE_OPENAI_GPT4o_DEPLOYMENT,
            "api_type": "azure",
            "api_key": AZURE_OPENAI_KEY,
            "base_url": AZURE_OPENAI_ENDPOINT,
            "api_version": AZURE_OPENAI_API_VERSION
        }
    ]
}



In [10]:
from azure.search.documents.models import VectorizedQuery
credential = AzureKeyCredential(key)
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

# Generate Document Embeddings using OpenAI Ada Model
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
# Function to generate embeddings for title and content fields, also used for query embeddings
def calc_embeddings(text):
    # model = "deployment_name"
    embeddings = aoai_client.embeddings.create(input = [text], model=AZURE_OPENAI_EMBEDDINGS_ADA_DEPLOYMENT_NAME).data[0].embedding
    return embeddings

def do_search(query: Annotated[str, "a query"])-> Annotated[str, "results"]:
    fields = "embedding"
    embedding = calc_embeddings(query)
    vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields=fields)
    # vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=1, fields=fields, exhaustive=True)
    results = search_client.search(  
        search_text=None,  
        vector_queries= [vector_query],
        select=["content"],
    )  
    answer = ''
    for result in results:  
        print(f"Score: {result['@search.score']}")  
        print(f"Content: {result['content']}")  
        answer = answer + result['content']
    return answer

In [11]:
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="A human admin. Interact with the WebSearchEngineer and AzureAISearcher and decides who to assign the task to.",
    code_execution_config={
        "work_dir": "code",
        "use_docker": False
    },
    human_input_mode="TERMINATE",
)

web_search_engineer = autogen.AssistantAgent(
    name="WebSearchEngineer",
    llm_config=llm_config,
    system_message="""Web Search Engineer. Make sure you save code to disk.  You write python/shell 
    code to solve tasks. Wrap the code in a code block that specifies the script type and the name of the file to 
    save to disk.""",
)

aia_searcher = autogen.AssistantAgent(
    name="AzureAISearcher",
    llm_config=llm_config,
    system_message="""
        Assistant who has extra content retrieval power for questions about the Moby Dick's book, 
        only use the functions you have been provided with. 
        Reply TERMINATE when the task is done.""",
)

# Register the tool signature with the assistant agent.
aia_searcher.register_for_llm(name="do_search", description="Get information about Moby Dick's book by Herman Melville.")(do_search)
# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="do_search")(do_search)


group_chat = autogen.GroupChat(
    agents=[user_proxy, web_search_engineer, aia_searcher], messages=[], max_round=12
)
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_config)

In [12]:
user_proxy.initiate_chat(
    manager,
    message="Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?",
)

Admin (to chat_manager):

Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?

--------------------------------------------------------------------------------

Next speaker: AzureAISearcher

AzureAISearcher (to chat_manager):

***** Suggested tool call (call_J0AeQfs5XZS8n3UH4oNfzJrD): do_search *****
Arguments: 
{"query":"Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?"}
**************************************************************************

--------------------------------------------------------------------------------

Next speaker: Admin


>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION do_search...
Score: 0.878709
Content: the studded iron-bound cask followed the sailor to the bottom, as if to 
yield him his pillow, though in sooth but a hard one. 
 
And thus the first man of the Pequod that mounted the mast to look out 
for the White Whale, on the White Whale’s own peculiar ground; 

ChatResult(chat_id=None, chat_history=[{'content': "Why does the coffin prepared for Queequeg become Ishmael's life buoy once the Pequod sinks?", 'role': 'assistant', 'name': 'Admin'}, {'content': 'None', 'tool_calls': [{'id': 'call_J0AeQfs5XZS8n3UH4oNfzJrD', 'function': {'arguments': '{"query":"Why does the coffin prepared for Queequeg become Ishmael\'s life buoy once the Pequod sinks?"}', 'name': 'do_search'}, 'type': 'function'}], 'name': 'AzureAISearcher', 'role': 'assistant'}, {'content': 'the studded iron-bound cask followed the sailor to the bottom, as if to \nyield him his pillow, though in sooth but a hard one. \n \nAnd thus the first man of the Pequod that mounted the mast to look out \nfor the White Whale, on the White Whale’s own peculiar ground; that man \nwas swallowed up in the deep. But few, perhaps, thought of that at the \ntime. Indeed, in some sort, they were not grieved at this event, at \nleast as a portent; for they regarded it, not as a foreshadowing of \nevil in

In [13]:
user_proxy.initiate_chat(
    manager,
    message="What is the weather today in Miami?",
)

Admin (to chat_manager):

What is the weather today in Miami?

--------------------------------------------------------------------------------

Next speaker: WebSearchEngineer

WebSearchEngineer (to chat_manager):

I'm unable to browse the internet in real-time to provide current weather updates. However, you can check the latest weather in Miami by visiting a weather website like Weather.com, AccuWeather, or using a weather app on your smartphone.

--------------------------------------------------------------------------------

Next speaker: Admin


>>>>>>>> USING AUTO REPLY...
Admin (to chat_manager):



--------------------------------------------------------------------------------

Next speaker: WebSearchEngineer

WebSearchEngineer (to chat_manager):

It looks like you didn't provide any additional information or a new task. How can I assist you further?

--------------------------------------------------------------------------------

Next speaker: WebSearchEngineer

WebSearchE

ChatResult(chat_id=None, chat_history=[{'content': 'What is the weather today in Miami?', 'role': 'assistant', 'name': 'Admin'}, {'content': "I'm unable to browse the internet in real-time to provide current weather updates. However, you can check the latest weather in Miami by visiting a weather website like Weather.com, AccuWeather, or using a weather app on your smartphone.", 'name': 'WebSearchEngineer', 'role': 'user'}, {'content': '', 'role': 'assistant', 'name': 'Admin'}, {'content': "It looks like you didn't provide any additional information or a new task. How can I assist you further?", 'name': 'WebSearchEngineer', 'role': 'user'}, {'content': "It seems like you didn't provide any additional information or a new task. How can I assist you further?", 'name': 'WebSearchEngineer', 'role': 'user'}, {'content': '', 'role': 'assistant', 'name': 'Admin'}, {'content': "It seems like you haven't provided any additional information or a new task. How can I assist you further?", 'name': 